In [31]:
import os
import time
import warnings
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm
from datetime import datetime

In [ ]:
# API KEY WeatherAPI = c2436f127ed34680b04182951232210

# Armado de la URL

In [11]:
# variable que almacena la ciudad que queremos consultar
query = 'Santiago'

# La key que nos asigne Weather api
api_key = 'c2436f127ed34680b04182951232210'

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
print(url_clima)

http://api.weatherapi.com/v1/forecast.json?key=c2436f127ed34680b04182951232210&q=Santiago&days=1&aqi=no&alerts=no


In [12]:
response = requests.get(url_clima).json()
response

{'location': {'name': 'Santiago',
  'region': 'Region Metropolitana',
  'country': 'Chile',
  'lat': -33.45,
  'lon': -70.67,
  'tz_id': 'America/Santiago',
  'localtime_epoch': 1699135168,
  'localtime': '2023-11-04 18:59'},
 'current': {'last_updated_epoch': 1699134300,
  'last_updated': '2023-11-04 18:45',
  'temp_c': 21.0,
  'temp_f': 69.8,
  'is_day': 1,
  'condition': {'text': 'Sunny',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
   'code': 1000},
  'wind_mph': 19.2,
  'wind_kph': 31.0,
  'wind_degree': 220,
  'wind_dir': 'SW',
  'pressure_mb': 1012.0,
  'pressure_in': 29.88,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 46,
  'cloud': 0,
  'feelslike_c': 21.0,
  'feelslike_f': 69.8,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 6.0,
  'gust_mph': 9.8,
  'gust_kph': 15.7},
 'forecast': {'forecastday': [{'date': '2023-11-04',
    'date_epoch': 1699056000,
    'day': {'maxtemp_c': 25.1,
     'maxtemp_f': 77.1,
     'mintemp_c': 11.2,
     'mintemp_f': 52.1,


In [13]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [14]:
# Usare solo Forecast para el entregable
# Obtenemos las keys para el pronostico del dia
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [29]:
# compruebo que el largo de la 'hora' sea de 24, asi me aseguro que informara en intervalos de 1 hora
# de 00:00 hrs a 23:00 hrs
len(response['forecast']['forecastday'][0]['hour'])

24

In [30]:
# FECHA
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[0]

'2023-11-04'

In [17]:
# HORA, como una variable entera
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0])

1

In [18]:
# Condicion del clima
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']

'Clear'

In [19]:
# TEMPERATURA en grados Celsius
response['forecast']['forecastday'][0]['hour'][0]['temp_c']

13.3

In [20]:
# ¿LLovera o no llovera?
# 0 : No lloverá ; 1: Lloverá
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [21]:
# Probabilidad de lluvia
response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain']

0

# Estructuración Dataframe

In [22]:
# Funcion que nos va a permitir extraer los campos de los 24 registros de cada hora de la api (weather api)

def get_forecast(response, i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha, hora, condicion, tempe, rain, prob_rain

In [23]:
# lista que recibira los registros
datos = []
for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    datos.append(get_forecast(response, i))

100%|██████████| 24/24 [00:00<?, ?it/s]


In [16]:
# Comprobamos los registros
datos

[('2023-11-04', 0, 'Clear', 13.3, 0, 0),
 ('2023-11-04', 1, 'Clear', 13.0, 0, 0),
 ('2023-11-04', 2, 'Clear', 12.6, 0, 0),
 ('2023-11-04', 3, 'Clear', 12.2, 0, 0),
 ('2023-11-04', 4, 'Clear', 11.9, 0, 0),
 ('2023-11-04', 5, 'Clear', 11.4, 0, 0),
 ('2023-11-04', 6, 'Clear', 11.2, 0, 0),
 ('2023-11-04', 7, 'Sunny', 16.3, 0, 0),
 ('2023-11-04', 8, 'Sunny', 19.2, 0, 0),
 ('2023-11-04', 9, 'Sunny', 20.8, 0, 0),
 ('2023-11-04', 10, 'Sunny', 22.2, 0, 0),
 ('2023-11-04', 11, 'Sunny', 23.3, 0, 0),
 ('2023-11-04', 12, 'Sunny', 24.3, 0, 0),
 ('2023-11-04', 13, 'Sunny', 24.9, 0, 0),
 ('2023-11-04', 14, 'Sunny', 25.1, 0, 0),
 ('2023-11-04', 15, 'Sunny', 24.4, 0, 0),
 ('2023-11-04', 16, 'Patchy rain possible', 23.2, 1, 86),
 ('2023-11-04', 17, 'Sunny', 21.9, 0, 0),
 ('2023-11-04', 18, 'Sunny', 21.0, 0, 0),
 ('2023-11-04', 19, 'Sunny', 18.0, 0, 0),
 ('2023-11-04', 20, 'Sunny', 16.0, 0, 0),
 ('2023-11-04', 21, 'Clear', 15.1, 0, 0),
 ('2023-11-04', 22, 'Clear', 13.7, 0, 0),
 ('2023-11-04', 23, 'Clear',

In [24]:
# Columnas del Dataframe
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','Prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,Prob_lluvia
0,2023-11-04,0,Clear,13.3,0,0
1,2023-11-04,1,Clear,13.0,0,0
2,2023-11-04,2,Clear,12.6,0,0
3,2023-11-04,3,Clear,12.2,0,0
4,2023-11-04,4,Clear,11.9,0,0
5,2023-11-04,5,Clear,11.4,0,0
6,2023-11-04,6,Clear,11.2,0,0
7,2023-11-04,7,Sunny,16.3,0,0
8,2023-11-04,8,Sunny,19.2,0,0
9,2023-11-04,9,Sunny,20.8,0,0


In [47]:
# Definir la llave primaria, llave de orden (sortkey) y llave de distribución (distkey)
"""
        -- Definir la clave primaria
        ALTER TABLE {table_name}
        ADD CONSTRAINT pk_{table_name}
        PRIMARY KEY (Fecha, Hora);
        
        -- Definir la clave de orden (sortkey)
        ALTER TABLE {table_name}
        COMPOUND SORTKEY (Hora);
        
        -- Definir la clave de distribución (distkey)
        ALTER TABLE {table_name}
        DISTKEY (Fecha);
"""

'\n        -- Definir la clave primaria\n        ALTER TABLE {table_name}\n        ADD CONSTRAINT pk_{table_name}\n        PRIMARY KEY (Fecha, Hora);\n        \n        -- Definir la clave de orden (sortkey)\n        ALTER TABLE {table_name}\n        COMPOUND SORTKEY (Hora);\n        \n        -- Definir la clave de distribución (distkey)\n        ALTER TABLE {table_name}\n        DISTKEY (Fecha);\n'

### Justificación Llaves de orden (SOTRKEY), distribución (DISTKEY) y llave primaria (Primary Key):

-  Si se desea almacenar información de una API por la columna 'Hora', es una buena idea utilizar 'Hora' como clave de ordenamiento, esto permitirá que las filas estén físicamente ordenadas por hora en Redshift, lo que facilitará la recuperación de datos en orden cronológico.

- La elección de la clave de distribución depende de cómo se planea realizar consultas en la tabla y de la naturaleza de los datos. En general, es aconsejable distribuir los datos de manera que las consultas comunes puedan ser ejecutadas de manera eficiente, dado que mi patron de consulta podria ser por fecha, se podrías usar 'Fecha' como clave de distribución. Por otro lado, si las consultas suelen involucrar múltiples columnas en igual medida, se podría considerar una distribución uniforme, en este caso conviene elegir una columna con alta cardinalidad o una columna de tipo hash para distribuir de manera uniforme los datos en los nodos de Redshift.

- Se define la llave primaria como combinación de "Fecha" y "Hora" , por unicidad de filas, ya que son altamente únicas y rara vez dos eventos tendrán exactamente la misma fecha y hora. Esto asegura que cada fila en la tabla se pueda identificar de manera única mediante la clave primaria, lo que es fundamental para mantener la integridad de los datos y prevenir duplicados no deseados. Otra razon es que los datos representan eventos cronológicos o de tiempo, tener "Fecha" y "Hora" como parte de la clave primaria garantiza que los datos se almacenen y consulten en el orden correcto, lo que es esencial para análisis de series temporales y consultas basadas en el tiempo, en este caso el clima. Tambien, facilita la actualización y eliminación de registros: Con la clave primaria basada en "Fecha" y "Hora", es más fácil identificar y actualizar registros específicos, especialmente si se están registrando datos a intervalos regulares. También facilita la eliminación de registros obsoletos o no deseados. Por ultima, se pueden crear consultas eficientes, si las consultas a menudo involucran selección o agregación de datos en función de una marca de tiempo, tener "Fecha" y "Hora" como parte de la clave primaria puede acelerar estas operaciones y mejorar el rendimiento de las consultas.

## Conección Redshift

In [4]:
%pip install "Redshift_connector[full]" sqlalchemy-Redshift

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.24.4 which is incompatible.
ydata-profiling 4.2.0 requires numpy<1.24,>=1.16.0, but you have numpy 1.24.4 which is incompatible.



     ---------------------------------------- 0.0/68.1 kB ? eta -:--:--
     ------------------------------------ --- 61.4/68.1 kB 3.2 MB/s eta 0:00:01
     -------------------------------------- 68.1/68.1 kB 732.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/135.8 kB ? eta -:--:--
   ---------------------------------------  133.1/135.8 kB 7.7 MB/s eta 0:00:01
   ---------------------------------------- 135.8/135.8 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
    --------------------------------------- 0.2/11.3 MB 5.0 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/11.3 MB 6.4 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/11.3 MB 6.9 MB/s eta 0:00:02
   ---- ----------------------------------- 1.2/11.3 MB 7.8 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/11.3 MB 5.6 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/11.3 MB 5.7 MB/s eta 0:00:02
   -

In [34]:
# librerias
import sqlalchemy as sa
from configparser import ConfigParser

### Definimos las funciones para conectar

In [65]:
def build_conn_string(config_path, config_section):
    """
    Construye la cadena de conexión a la base de datos
    a partir de un archivo de configuración.
    """

    # Lee el archivo de configuración
    parser = ConfigParser()
    parser.read(config_path)

    # Lee la sección de configuración de PostgreSQL
    config = parser[config_section]
    host = config['host']
    port = config['port']
    dbname = config['dbname']
    username = config['username']
    pwd = config['pwd']

    # Construye la cadena de conexión
    conn_string = f'postgresql://{username}:{pwd}@{host}:{port}/{dbname}?sslmode=require'

    return conn_string

In [66]:
def connect_to_db(conn_string):
    """
    Crea una conexión a la base de datos.
    """
    engine = sa.create_engine(conn_string)
    conn = engine.connect()
    return conn, engine

## Preparación de Datos

In [69]:
pip install psycopg2


   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ----------------------------------- ---- 1.0/1.2 MB 31.7 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.2 MB 35.5 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.2 MB 35.5 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 7.3 MB/s eta 0:00:00


In [70]:
# Cargar el dataframe creado anteriormente(df) con los datos extraidos de la API
# Conexión a Redshift
import psycopg2

config_dir = "config/config.ini"
conn_string = build_conn_string(config_dir,"postgresql")
conn, engine = connect_to_db(conn_string)

In [71]:
# Cargar el dataframe(df) a la base de datos
dfs = [df]
tbl_names = ["santiago_clima"]

for df, tbl_name in zip(dfs, tbl_names):
    df.to_sql(
        name=tbl_name,
        con=conn,
        schema="stage",
        if_exists="replace",
        method="multi",
        index=False,
    )

ProgrammingError: (psycopg2.errors.InvalidSchemaName) schema "stage" does not exist

[SQL: 
CREATE TABLE stage.santiago_clima (
	"Fecha" TEXT, 
	"Hora" BIGINT, 
	"Condicion" TEXT, 
	"Temperatura" FLOAT(53), 
	"Lluvia" BIGINT, 
	"Prob_lluvia" BIGINT
)

]
(Background on this error at: https://sqlalche.me/e/14/f405)